SDBMS Battery Simulation

Install Pybamm on Jupyer

In [34]:
%pip install pybamm
%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Initiating PyBamm

In [35]:
import pybamm
from pybamm import BaseModel
from pprint import pprint
import numpy as np

Choose Battery Chemical & Model

In [36]:
## [1] Doyle Fuller newman [2] Single Particle Model (SPM) [3] Single Particle Model with Electrolyte (SPMe)
availableModel = ["DFN","SPM", "SPME"]
selectedModel = availableModel[0]
model : BaseModel

match selectedModel:
    case "DFN":
        model : BaseModel = pybamm.lithium_ion.DFN()
    case "SPM":
        model : BaseModel = pybamm.lithium_ion.SPM()
    case "SPME":
        model : BaseModel = pybamm.lithium_ion.SPMe()
        


Print Available Model Input Parameter.
Input Parameter = predefined config / parameter for physical, chemical and electrochemical properties of battery.

In [37]:
modelInputParameters = list(pybamm.parameter_sets)
pprint(modelInputParameters)

['Ai2020',
 'Chayambuka2022',
 'Chen2020',
 'Chen2020_composite',
 'ECM_Example',
 'Ecker2015',
 'Ecker2015_graphite_halfcell',
 'MSMR_Example',
 'Marquis2019',
 'Mohtat2020',
 'NCA_Kim2011',
 'OKane2022',
 'OKane2022_graphite_SiOx_halfcell',
 'ORegan2022',
 'Prada2013',
 'Ramadass2004',
 'Sulzer2019',
 'Xu2019']


Display Selected Model input parameter value

In [38]:
selectedInputParameter =  pybamm.ParameterValues("Chen2020")
inputBatteryCapacity = 5
selectedInputParameter.update({
    "Nominal cell capacity [A.h]": inputBatteryCapacity
})
## DFN => Chen2020, Ai2020. SPME = Marquis2019, SPM = Marquis2019
pprint(selectedInputParameter)

{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.2594,
 'Cell cooling surface area [m2]': 0.00531,
 'Cell thermal expansion coefficient [m.K-1]': 1.1e-06,
 'Cell volume [m3]': 2.42e-05,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 5.0,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Electrode height [m]': 0.065,
 'Electrode width [m]': 1.58,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Nyman2008 at 0x169f99940>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Nyman2008 at 0x169f99f80>,
 'Electron charge [C]': 1.602176634e-19,
 'Faraday constant [C.mol-1]': 96485.33212331001,
 'Ideal gas constant [J.K-1.mol-1]': 8.31446261815324,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 29866.0,
 'Initial 

Get Q (Battery Capacity)

In [39]:
batteryCapacity = selectedInputParameter['Nominal cell capacity [A.h]']
pprint(batteryCapacity)

5


Print Available Model Output Parameter

In [40]:
availableOutputModelVariable = model.variable_names()
#pprint(availableOutputModelVariable)

Available Instructions

- "Discharge at 1C for 0.5 hours",
- "Discharge at C/20 for 0.5 hours",
- "Charge at 0.5 C for 45 minutes",
- "Discharge at 1 A for 90 seconds",
- "Charge at 200mA for 45 minutes",
- "Discharge at 1 W for 0.5 hours",
- "Charge at 200 mW for 45 minutes",
- "Rest for 10 minutes",
- "Hold at 1 V for 20 seconds",
- "Charge at 1 C until 4.1V",
- "Hold at 4.1 V until 50 mA",
- #"Hold at 3V until C/50",

Run Simulation

In [41]:
experiment = pybamm.Experiment(
    [
        (
            "Charge at 1 C until 4.2 V",
            "Discharge at C/0.5 for 2 hours or until 2.5 V"
        )
    ],
    period="1 second"  # <-- This sets data points every 1 second
)
sim = pybamm.Simulation(model, experiment=experiment, parameter_values=selectedInputParameter)
sim.solve()

Visualize

Custom Plot with Selected Model Result Parameter

In [46]:
output_variables = ["Time [s]", "Current [A]", "Voltage [V]", "X-averaged cell temperature [K]"]
sim.plot(output_variables=output_variables)

interactive(children=(FloatSlider(value=0.0, description='t', max=1703.0718861513194, step=17.030718861513193)…

Store to pandas data frame and Compute SOC

In [47]:
import pandas as pd
output_variables.append("BatteryCapacity")
output_variables.append("TimeDiff")
output_variables.append("CapacityDiff")
output_variables.append("SOC")
data = {}
num_rows = len(sim.solution["Time [min]"].entries)

for var in output_variables:
    if var == "TimeDiff":
        timeDiffValues = []
        for i in range(num_rows):
            if i == 0:
                timeDiffValues.append(0)
            else:
                prevTime = sim.solution["Time [s]"].entries[i-1]
                currentTime = sim.solution["Time [s]"].entries[i]
                deltaTime = currentTime - prevTime
                timeDiffValues.append(deltaTime)
        data[var] = timeDiffValues
    elif var == "BatteryCapacity":
        data[var] = inputBatteryCapacity
    elif var == "CapacityDiff":
        capacityDiffValues = []
        for i in range(num_rows):
            if i == 0:
                capacityDiffValues.append(0)
            else:
                deltaCapacity =  sim.solution["Current [A]"].entries[i]*(timeDiffValues[i]/3600)*-1
                capacityDiffValues.append(deltaCapacity)
        data[var] = capacityDiffValues
    elif var == "SOC":
        soc_values = []
        for i in range(num_rows):
            if i == 0:
                soc_values.append(1)
            else:
                ## Compute using Coulomb Counting
                updatedSOC = soc_values[i-1]+(capacityDiffValues[i]/batteryCapacity)
                soc_values.append(updatedSOC)
        data[var] = soc_values
    else:
        data[var] = sim.solution[var].entries
        
simulationOutputDf = pd.DataFrame(data)
display(simulationOutputDf)
simulationOutputDf.to_csv("simulation_output_coulomb.csv", index=False)

,Time [s],Current [A],Voltage [V],X-averaged cell temperature [K],BatteryCapacity,TimeDiff,CapacityDiff,SOC
0,0.000000,10.0,3.965329,298.15,5,0.000000,0.000000,1.000000
1,1.000000,10.0,3.954918,298.15,5,1.000000,-0.002778,0.999444
2,2.000000,10.0,3.945408,298.15,5,1.000000,-0.002778,0.998889
3,3.000000,10.0,3.936768,298.15,5,1.000000,-0.002778,0.998333
4,4.000000,10.0,3.928941,298.15,5,1.000000,-0.002778,0.997778
...,...,...,...,...,...,...,...,...
1700,1700.000000,10.0,2.514492,298.15,5,1.000000,-0.002778,0.055556
1701,1701.000000,10.0,2.509855,298.15,5,1.000000,-0.002778,0.055000
1702,1702.000000,10.0,2.505144,298.15,5,1.000000,-0.002778,0.054444
1703,1703.000000,10.0,2.500357,298.15,5,1.000000,-0.002778,0.053889


Plot voltage component

In [25]:
pprint("end")

'end'
